In [20]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from nltk import ngrams
import re
import pandas as pd
from collections import Counter

In [2]:
# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Load stopwords
stop_words = set(stopwords.words('english'))

In [36]:
df = pd.read_csv("abstracts_stunting_in_children.csv", encoding='unicode_escape')
corpus = list(df['Abstract'])
word_search = pd.read_csv("Word_search_nfhs_cnns.csv")
search_word_nfhs = list(word_search['search_word_nfhs'])
search_word_with_underscores = [sentence.replace(" ", "_") for sentence in search_word_nfhs]

In [5]:
# Text preprocessing function
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    #tokens = [word for word in tokens if word not in stop_words]
    bigram_corpus = list(ngrams(text.split(), 2))
    bigram_corpus_lists = [list(subset) for subset in bigram_corpus]
    return bigram_corpus_lists

In [6]:
# Tokenize the corpus
tokenized_corpus = [preprocess_text(sentence) for sentence in corpus]

In [7]:
# Flatten the nested list of bigrams into a list of sentences
tokenized_corpus_sentences = [word for sublist in tokenized_corpus for word in sublist]

In [8]:
# Convert corpus into bigrams
bigram_model = Phrases(tokenized_corpus_sentences, min_count=1, threshold=1)
bigram_model_phraser = Phraser(bigram_model)

In [13]:
# Apply Phraser to nested list
bigram_corpus_phrased = [bigram_model_phraser[sentence] for sentence in tokenized_corpus]

# Flatten any nested lists in bigram_corpus_phrased
flattened_phrased_corpus = [word for sublist in bigram_corpus_phrased for word in sublist]

In [14]:
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=flattened_phrased_corpus, vector_size=100, window=5, min_count=1, workers=4)

# Access word vectors
word_vectors = word2vec_model.wv

In [38]:
# Function to find the co-occurrence rank for two given words
def co_occurrence_rank(word1, word2, corpus):
    try:
        # Count co-occurrences of word1 and word2
        co_occurrences = Counter()
        for sentence in corpus:
            if word1 in sentence and word2 in sentence:
                co_occurrences.update(sentence)
        
        # Sort by co-occurrence count
        sorted_vocab = sorted(model.wv.index_to_key, key=lambda x: co_occurrences[x], reverse=True)
        
        # Find the ranks of word1 and word2
        rank_word1 = sorted_vocab.index(word1) + 1
        rank_word2 = sorted_vocab.index(word2) + 1
        
        return rank_word1, rank_word2
    except ValueError:
        print(f"Words: {word1}, {word2} - null")
        return None, None

In [ ]:
# Example usage
word1 = "cat"
word2 = "dog"
rank1, rank2 = co_occurrence_rank(word1, word2, bigram_corpus_phrased)

In [ ]:

print(f"Co-occurrence rank of '{word1}': {rank1}")
print(f"Co-occurrence rank of '{word2}': {rank2}")

In [39]:
combs = combinations(search_word_with_underscores, 2)
keywords_counts = {}
for c in combs:
    rank1, rank2 = co_occurrence_rank(c[0],c[1],bigram_corpus_phrased)
    print(f"Co-occurrence rank of '{c[0]}': '{rank1}' and '{c[1]}': {rank2}")

Words: educated, child_birth - null
Co-occurrence rank of 'educated': 'None' and 'child_birth': None
Words: educated, electricity - null
Co-occurrence rank of 'educated': 'None' and 'electricity': None
Words: educated, drinking_water - null
Co-occurrence rank of 'educated': 'None' and 'drinking_water': None
Words: educated, sanitation_facility - null
Co-occurrence rank of 'educated': 'None' and 'sanitation_facility': None
Words: educated, clean_cooking_fuel - null
Co-occurrence rank of 'educated': 'None' and 'clean_cooking_fuel': None
Words: educated, iodized_salt - null
Co-occurrence rank of 'educated': 'None' and 'iodized_salt': None
Words: educated, health_insurance - null
Co-occurrence rank of 'educated': 'None' and 'health_insurance': None
Words: educated, attended_school - null
Co-occurrence rank of 'educated': 'None' and 'attended_school': None
Words: educated, illiterate - null
Co-occurrence rank of 'educated': 'None' and 'illiterate': None
Words: educated, birth_order - null
C

Words: child_birth, zinc - null
Co-occurrence rank of 'child_birth': 'None' and 'zinc': None
Words: child_birth, diarrhoea - null
Co-occurrence rank of 'child_birth': 'None' and 'diarrhoea': None
Words: child_birth, respiratory_infection - null
Co-occurrence rank of 'child_birth': 'None' and 'respiratory_infection': None
Words: child_birth, fever - null
Co-occurrence rank of 'child_birth': 'None' and 'fever': None
Words: child_birth, breastfeeding - null
Co-occurrence rank of 'child_birth': 'None' and 'breastfeeding': None
Words: child_birth, breastfed - null
Co-occurrence rank of 'child_birth': 'None' and 'breastfed': None
Words: child_birth, solid_food - null
Co-occurrence rank of 'child_birth': 'None' and 'solid_food': None
Words: child_birth, adequate_diet - null
Co-occurrence rank of 'child_birth': 'None' and 'adequate_diet': None
Words: child_birth, stunted - null
Co-occurrence rank of 'child_birth': 'None' and 'stunted': None
Words: child_birth, wasted - null
Co-occurrence rank 

Words: drinking_water, iodized_salt - null
Co-occurrence rank of 'drinking_water': 'None' and 'iodized_salt': None
Words: drinking_water, health_insurance - null
Co-occurrence rank of 'drinking_water': 'None' and 'health_insurance': None
Words: drinking_water, attended_school - null
Co-occurrence rank of 'drinking_water': 'None' and 'attended_school': None
Words: drinking_water, illiterate - null
Co-occurrence rank of 'drinking_water': 'None' and 'illiterate': None
Words: drinking_water, birth_order - null
Co-occurrence rank of 'drinking_water': 'None' and 'birth_order': None
Words: drinking_water, mother_pregnant - null
Co-occurrence rank of 'drinking_water': 'None' and 'mother_pregnant': None
Words: drinking_water, hygienic_period - null
Co-occurrence rank of 'drinking_water': 'None' and 'hygienic_period': None
Words: drinking_water, family_planning - null
Co-occurrence rank of 'drinking_water': 'None' and 'family_planning': None
Words: drinking_water, female_sterilization - null
Co-

Words: sanitation_facility, oral_rehydration - null
Co-occurrence rank of 'sanitation_facility': 'None' and 'oral_rehydration': None
Words: sanitation_facility, zinc - null
Co-occurrence rank of 'sanitation_facility': 'None' and 'zinc': None
Words: sanitation_facility, diarrhoea - null
Co-occurrence rank of 'sanitation_facility': 'None' and 'diarrhoea': None
Words: sanitation_facility, respiratory_infection - null
Co-occurrence rank of 'sanitation_facility': 'None' and 'respiratory_infection': None
Words: sanitation_facility, fever - null
Co-occurrence rank of 'sanitation_facility': 'None' and 'fever': None
Words: sanitation_facility, breastfeeding - null
Co-occurrence rank of 'sanitation_facility': 'None' and 'breastfeeding': None
Words: sanitation_facility, breastfed - null
Co-occurrence rank of 'sanitation_facility': 'None' and 'breastfed': None
Words: sanitation_facility, solid_food - null
Co-occurrence rank of 'sanitation_facility': 'None' and 'solid_food': None
Words: sanitation_

Words: clean_cooking_fuel, blood_sugar - null
Co-occurrence rank of 'clean_cooking_fuel': 'None' and 'blood_sugar': None
Words: clean_cooking_fuel, blood_pressure - null
Co-occurrence rank of 'clean_cooking_fuel': 'None' and 'blood_pressure': None
Words: clean_cooking_fuel, tobacco - null
Co-occurrence rank of 'clean_cooking_fuel': 'None' and 'tobacco': None
Words: clean_cooking_fuel, alcohol - null
Co-occurrence rank of 'clean_cooking_fuel': 'None' and 'alcohol': None
Words: iodized_salt, health_insurance - null
Co-occurrence rank of 'iodized_salt': 'None' and 'health_insurance': None
Words: iodized_salt, attended_school - null
Co-occurrence rank of 'iodized_salt': 'None' and 'attended_school': None
Words: iodized_salt, illiterate - null
Co-occurrence rank of 'iodized_salt': 'None' and 'illiterate': None
Words: iodized_salt, birth_order - null
Co-occurrence rank of 'iodized_salt': 'None' and 'birth_order': None
Words: iodized_salt, mother_pregnant - null
Co-occurrence rank of 'iodized

...

Words: institutional_birth, zinc - null
Co-occurrence rank of 'institutional_birth': 'None' and 'zinc': None
Words: institutional_birth, diarrhoea - null
Co-occurrence rank of 'institutional_birth': 'None' and 'diarrhoea': None
Words: institutional_birth, respiratory_infection - null
Co-occurrence rank of 'institutional_birth': 'None' and 'respiratory_infection': None
Words: institutional_birth, fever - null
Co-occurrence rank of 'institutional_birth': 'None' and 'fever': None
Words: institutional_birth, breastfeeding - null
Co-occurrence rank of 'institutional_birth': 'None' and 'breastfeeding': None
Words: institutional_birth, breastfed - null
Co-occurrence rank of 'institutional_birth': 'None' and 'breastfed': None
Words: institutional_birth, solid_food - null
Co-occurrence rank of 'institutional_birth': 'None' and 'solid_food': None
Words: institutional_birth, adequate_diet - null
Co-occurrence rank of 'institutional_birth': 'None' and 'adequate_diet': None
Words: institutional_bir

Words: diarrhoea, solid_food - null
Co-occurrence rank of 'diarrhoea': 'None' and 'solid_food': None
Words: diarrhoea, adequate_diet - null
Co-occurrence rank of 'diarrhoea': 'None' and 'adequate_diet': None
Words: diarrhoea, stunted - null
Co-occurrence rank of 'diarrhoea': 'None' and 'stunted': None
Words: diarrhoea, wasted - null
Co-occurrence rank of 'diarrhoea': 'None' and 'wasted': None
Words: diarrhoea, severely_wasted - null
Co-occurrence rank of 'diarrhoea': 'None' and 'severely_wasted': None
Words: diarrhoea, underweight - null
Co-occurrence rank of 'diarrhoea': 'None' and 'underweight': None
Words: diarrhoea, overweight - null
Co-occurrence rank of 'diarrhoea': 'None' and 'overweight': None
Words: diarrhoea, mother_BMI - null
Co-occurrence rank of 'diarrhoea': 'None' and 'mother_BMI': None
Words: diarrhoea, mother_weight - null
Co-occurrence rank of 'diarrhoea': 'None' and 'mother_weight': None
Words: diarrhoea, waist_ratio - null
Co-occurrence rank of 'diarrhoea': 'None' an

Words: breastfeeding, tobacco - null
Co-occurrence rank of 'breastfeeding': 'None' and 'tobacco': None
Words: breastfeeding, alcohol - null
Co-occurrence rank of 'breastfeeding': 'None' and 'alcohol': None
Words: breastfed, solid_food - null
Co-occurrence rank of 'breastfed': 'None' and 'solid_food': None
Words: breastfed, adequate_diet - null
Co-occurrence rank of 'breastfed': 'None' and 'adequate_diet': None
Words: breastfed, stunted - null
Co-occurrence rank of 'breastfed': 'None' and 'stunted': None
Words: breastfed, wasted - null
Co-occurrence rank of 'breastfed': 'None' and 'wasted': None
Words: breastfed, severely_wasted - null
Co-occurrence rank of 'breastfed': 'None' and 'severely_wasted': None
Words: breastfed, underweight - null
Co-occurrence rank of 'breastfed': 'None' and 'underweight': None
Words: breastfed, overweight - null
Co-occurrence rank of 'breastfed': 'None' and 'overweight': None
Words: breastfed, mother_BMI - null
Co-occurrence rank of 'breastfed': 'None' and '

Words: wasted, alcohol - null
Co-occurrence rank of 'wasted': 'None' and 'alcohol': None
Words: severely_wasted, underweight - null
Co-occurrence rank of 'severely_wasted': 'None' and 'underweight': None
Words: severely_wasted, overweight - null
Co-occurrence rank of 'severely_wasted': 'None' and 'overweight': None
Words: severely_wasted, mother_BMI - null
Co-occurrence rank of 'severely_wasted': 'None' and 'mother_BMI': None
Words: severely_wasted, mother_weight - null
Co-occurrence rank of 'severely_wasted': 'None' and 'mother_weight': None
Words: severely_wasted, waist_ratio - null
Co-occurrence rank of 'severely_wasted': 'None' and 'waist_ratio': None
Words: severely_wasted, hip_ratio - null
Co-occurrence rank of 'severely_wasted': 'None' and 'hip_ratio': None
Words: severely_wasted, anaemic - null
Co-occurrence rank of 'severely_wasted': 'None' and 'anaemic': None
Words: severely_wasted, mother_anaemic - null
Co-occurrence rank of 'severely_wasted': 'None' and 'mother_anaemic': No

Words: mother_anaemic, blood_sugar - null
Co-occurrence rank of 'mother_anaemic': 'None' and 'blood_sugar': None
Words: mother_anaemic, blood_pressure - null
Co-occurrence rank of 'mother_anaemic': 'None' and 'blood_pressure': None
Words: mother_anaemic, tobacco - null
Co-occurrence rank of 'mother_anaemic': 'None' and 'tobacco': None
Words: mother_anaemic, alcohol - null
Co-occurrence rank of 'mother_anaemic': 'None' and 'alcohol': None
Words: pregnant_anaemic, blood_sugar - null
Co-occurrence rank of 'pregnant_anaemic': 'None' and 'blood_sugar': None
Words: pregnant_anaemic, blood_pressure - null
Co-occurrence rank of 'pregnant_anaemic': 'None' and 'blood_pressure': None
Words: pregnant_anaemic, tobacco - null
Co-occurrence rank of 'pregnant_anaemic': 'None' and 'tobacco': None
Words: pregnant_anaemic, alcohol - null
Co-occurrence rank of 'pregnant_anaemic': 'None' and 'alcohol': None
Words: blood_sugar, blood_pressure - null
Co-occurrence rank of 'blood_sugar': 'None' and 'blood_pre